In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
def random_split(lis,percent):
    import random
    
    l1 = []
    l2 = []
    sample=random.sample(range(len(lis)), int(len(lis)*percent))
    for i in sample :
            l1.append(lis[i])
        

    for i in range(len(lis)) : 
        if i not in sample :
            l2.append(lis[i])

    
        
    return l1,l2        

In [4]:
import torch
"""
class Face_dataset(Dataset):

    def __init__ (self,dir_path,photo_list,transform=None):
        super().__init__()
        self.transform = transform

        self.y = [ int(x.split("_")[0]) for x in photo_list]
        self.x = []
        self.len = len(self.y)
      
        self.y = torch.LongTensor(self.y)
        
        for photo in photo_list :
            self.x.append(self.transform(Image.open(dir_path+"/"+photo).convert('RGB')))

    def __getitem__ (self,index):
        return (self.x[index],self.y[index])
    
    def __len__ (self):
        return self.len
"""
class Face_dataset(Dataset):

    def __init__ (self,df,transform=None):
        super().__init__()
        self.transform = transform

        self.y = list(df["Label"].astype(int))
        self.x = list(df["Filepath"])
        self.photos = []
        print(1)
        self.len = len(self.y)
      
        self.y = torch.LongTensor(self.y)
        
        for photo in self.x :
            self.photos.append(self.transform(Image.open(photo).convert('RGB')))

    def __getitem__ (self,index):
        return (self.photos[index],self.y[index])
    
    def __len__ (self):
        return self.len
"""
class Face_dataset(Dataset):

    def __init__ (self,dir_path,photo_list,transform=None):
        super().__init__()
        self.transform = transform

        self.y = [ int(x.split("_")[0]) for x in photo_list]
        self.x = []
        self.len = len(self.y)
      
        self.y = torch.LongTensor(self.y)
        
        for photo in photo_list :
            self.x.append(dir_path+"/"+photo)

    def __getitem__ (self,index):
        return (self.transform(Image.open(self.x[index]).convert('RGB')),self.y[index])
    
    def __len__ (self):
        return self.len
"""

'\nclass Face_dataset(Dataset):\n\n    def __init__ (self,dir_path,photo_list,transform=None):\n        super().__init__()\n        self.transform = transform\n\n        self.y = [ int(x.split("_")[0]) for x in photo_list]\n        self.x = []\n        self.len = len(self.y)\n      \n        self.y = torch.LongTensor(self.y)\n        \n        for photo in photo_list :\n            self.x.append(dir_path+"/"+photo)\n\n    def __getitem__ (self,index):\n        return (self.transform(Image.open(self.x[index]).convert(\'RGB\')),self.y[index])\n    \n    def __len__ (self):\n        return self.len\n'

In [6]:
photo_list = os.listdir("/content/drive/MyDrive/raice_detect/UTKFace")




Filepath                                                                            Label
/content/drive/MyDrive/raice_detect/UTKFace/100_0_0_20170112215240346.jpg.chip.jpg  0        1
/content/drive/MyDrive/raice_detect/UTKFace/36_1_4_20170105164808838.jpg.chip.jpg   4        1
/content/drive/MyDrive/raice_detect/UTKFace/36_1_1_20170113001658573.jpg.chip.jpg   1        1
/content/drive/MyDrive/raice_detect/UTKFace/36_1_1_20170116014305365.jpg.chip.jpg   1        1
/content/drive/MyDrive/raice_detect/UTKFace/36_1_1_20170116161213859.jpg.chip.jpg   1        1
                                                                                            ..
/content/drive/MyDrive/raice_detect/UTKFace/26_1_1_20170116164012804.jpg.chip.jpg   1        1
/content/drive/MyDrive/raice_detect/UTKFace/26_1_1_20170116171052794.jpg.chip.jpg   1        1
/content/drive/MyDrive/raice_detect/UTKFace/26_1_1_20170116171823756.jpg.chip.jpg   1        1
/content/drive/MyDrive/raice_detect/UTKFace/26_1_1_2017

In [17]:
filepaths = []
for photo in photo_list :
    filepaths.append("/content/drive/MyDrive/raice_detect/UTKFace"+"/"+photo)

filepaths = pd.Series(filepaths, name='Filepath').astype(str)

labels = ((x.split("_")[2]) for x in photo_list)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1)
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x == '20170116174525125.jpg.chip.jpg' else x)
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x == '4' else x)



image_df = image_df.dropna(axis=0)

samples = []

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(3000, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
image_df["Label"].value_counts()

0    3000
1    3000
2    3000
3    3000
Name: Label, dtype: int64

In [18]:
train_transform = transforms.Compose(
    [
    transforms.Resize((64,64)),
    
    transforms.ToTensor(),
     
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])



train_df, temp_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

test_df, v_df = train_test_split(temp_df, train_size=0.5, shuffle=True, random_state=1)


#train_list,test_list = random_split(photo_list,0.6)
#valid_list = train_list[:int(len(train_list)*0.3)]

train_dataset = Face_dataset(train_df,train_transform)
test_dataset = Face_dataset(test_df,train_transform)
valid_dataset = Face_dataset(v_df,train_transform)

train_loader = DataLoader (train_dataset,batch_size = 32 ,shuffle = True)
test_loader = DataLoader (test_dataset,batch_size = 32 ,shuffle = True)
valid_loader = DataLoader (valid_dataset,batch_size = 32 ,shuffle = True)

#image_df["Label"].value_counts()

1
1
1


In [33]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class Cnn(nn.Module):
    def __init__ (self):
        super().__init__()
        # 64*64*3
        
        self.conv1 = nn.Conv2d(3,16,3,padding = 1)
        self.conv2 = nn.Conv2d(16,16,3,padding = 1)
        self.conv3 = nn.Conv2d(16,16,3,padding = 1)
        
        self.batch1 = nn.BatchNorm2d(16)
        #---------------------------------
        self.conv4 = nn.Conv2d(16,32,3,padding = 1)
        self.conv5 = nn.Conv2d(32,32,3,padding = 1)
        self.conv6 = nn.Conv2d(32,32,3,padding = 1)
        
        self.batch2 = nn.BatchNorm2d(32)
        #---------------------------------        
        self.conv7 = nn.Conv2d(32,64,3,padding = 1)
        self.conv8 = nn.Conv2d(64,64,3,padding = 1)
        self.conv9 = nn.Conv2d(64,64,3,padding = 1)
        
        self.batch3 = nn.BatchNorm2d(64)
        #---------------------------------        
        self.max = nn.MaxPool2d(2,2)
        
        self.drop = nn.Dropout(0.5)
        
        #8*8*64
        #---------------------------------
        self.flat = 8*8*64
        self.leyers = [2000,1500,500,300,150,117]
        self.fc1 = nn.Linear(self.flat, self.leyers[0])
        self.fc2 = nn.Linear(self.leyers[0] , self.leyers[1])
        self.fc3 = nn.Linear(self.leyers[1],self.leyers[2])
        self.fc4 = nn.Linear(self.leyers[2],self.leyers[3])
        self.fc5 = nn.Linear(self.leyers[3],self.leyers[4])
        self.fc6 = nn.Linear(self.leyers[4],self.leyers[5])
        
    def  Loss_Function (self,lr):
        
        self.criterion = nn.CrossEntropyLoss() 
        self.optimizer = optim.Adam(self.parameters(),lr)
        
        
    def forward (self,x):
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.drop(x)
        x = self.max(x)
        x = self.batch1(x)
        
        
        #--------------
        
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.drop(x)
        x = F.relu(self.conv6(x))
        x = self.max(x)
        x = self.batch2(x)
        
        #--------------
        
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = self.drop(x)
        x = F.relu(self.conv9(x))
        x = self.max(x)
        x = self.batch3(x)
        
        #--------------
        
        x = x.view(-1,self.flat)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = F.relu(self.fc4(x))
        x =  F.relu(self.fc5(x))
        x = self.fc6(x)
        return x
    
    def trains_(self,epoch,lr,train_data,valid_data):
    
        self.Loss_Function(lr)
        valid_min = np.Inf
        for _ in range(epoch+1):
            t_loss = 0
            v_loss = 0
            self.train()
            for x,y in train_data:
                
                self.optimizer.zero_grad()
                
                output = self.predict(x)
                loss = self.criterion(output,y)
                loss.backward()
                self.optimizer.step()
                t_loss+=loss.item()
            self.eval()
            for x,y in valid_data:
                
                
                output = self.predict(x)
                loss = self.criterion(output,y)
                loss.backward()
                v_loss+=loss.item()
                
            t_loss = t_loss/len(train_data.dataset)
            v_loss = v_loss/len(valid_data.dataset)
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            _, t_loss, v_loss))
            
            if v_loss <= valid_min:
                
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_min,
                v_loss))
                torch.save(self.state_dict(), 'model.pt')
                valid_min = v_loss
                
            
    def predict(self,x):
        
        output = self.forward(x)
        
        return output
    
    
    def test(self,test_data):
        valid_min = np.Inf
        for _ in range(100+1):
          t_loss = 0
          v_loss = 0
      
          for x,y in test_data:
              output = self.predict(x)
              loss = self.criterion(output,y)
              loss.backward()
              v_loss+=loss.item()

          v_loss = v_loss/len(test_data.dataset)
          print('Epoch:  Validation Loss: {:.6f}'.format(
           v_loss))
          


In [35]:
from os.path import exists

model = Cnn()
if(exists("/content/model.pt")):
  model.load_state_dict(torch.load('model.pt'))
else:
  model.trains_(50,0.00005,train_loader,valid_loader)

Epoch: 0 	Training Loss: 0.056070 	Validation Loss: 0.042454
Validation loss decreased (inf --> 0.042454).  Saving model ...
Epoch: 1 	Training Loss: 0.036222 	Validation Loss: 0.041212
Validation loss decreased (0.042454 --> 0.041212).  Saving model ...
Epoch: 2 	Training Loss: 0.027598 	Validation Loss: 0.037568
Validation loss decreased (0.041212 --> 0.037568).  Saving model ...
Epoch: 3 	Training Loss: 0.024644 	Validation Loss: 0.037516
Validation loss decreased (0.037568 --> 0.037516).  Saving model ...
Epoch: 4 	Training Loss: 0.022878 	Validation Loss: 0.033221
Validation loss decreased (0.037516 --> 0.033221).  Saving model ...
Epoch: 5 	Training Loss: 0.021518 	Validation Loss: 0.026767
Validation loss decreased (0.033221 --> 0.026767).  Saving model ...
Epoch: 6 	Training Loss: 0.020636 	Validation Loss: 0.030310
Epoch: 7 	Training Loss: 0.019763 	Validation Loss: 0.030098
Epoch: 8 	Training Loss: 0.019033 	Validation Loss: 0.029464
Epoch: 9 	Training Loss: 0.018431 	Validat

KeyboardInterrupt: ignored

In [49]:
def check_Accuracy(model,test_data):
  acc = 0
  correct = 0
  total = 0
  for x,y in test_data:
    predict = model.predict(x)
    pre=predict.data.max(1, keepdim=True)[1]
    correct += np.sum(np.squeeze(pre.eq(y.data.view_as(pre))).cpu().numpy())
    total += x.size(0)

  print('\nTest Accuracy: %2d%% (%2d/%2d)' % (100. * correct / total, correct, total))

In [50]:
for _ in range(5):
  print(check_Accuracy(model,test_loader))


Test Accuracy: 78% (1414/1800)
None

Test Accuracy: 77% (1396/1800)
None

Test Accuracy: 79% (1423/1800)
None

Test Accuracy: 78% (1405/1800)
None

Test Accuracy: 77% (1396/1800)
None
